In [ ]:
%load_ext autoreload
%autoreload 2

import molpy as mp
import numpy as np
from pathlib import Path

data_path = Path("data/case2")

In [ ]:
typifier = mp.typifier.AmberToolsTypifier("gaff", "bcc")
reacter = mp.reacter.LammpsReacter(
    mp.typifier.AmberToolsTypifier("gaff", "bcc"),
)

struct_hd = mp.Struct.from_frame(
    mp.io.read_pdb(data_path / "hexamethylenediamine.pdb"), "hd"
)
struct_aa = mp.Struct.from_frame(mp.io.read_pdb(data_path / "adipic_acid.pdb"), "aa")

struct_hd = typifier.typify(struct_hd)
struct_aa = typifier.typify(struct_aa)

In [ ]:
init1 = struct_hd.atoms[0]
init2 = struct_aa.atoms[0]
end1 = struct_hd.atoms[8]
end2 = struct_aa.atoms[2]

# find end groups
hd_end = reacter.find_template(
    "hd_end",
    struct_hd,
    end1, init1
)
aa_end = reacter.find_template(
    "aa_end",
    struct_aa,
    end2, init2,
)

stp1_react = reacter.react(
    "stp1_react",
    mp.Struct.concat("stp1_unreact", [hd_end, aa_end]),
    init1, init2,
    end1, end2,
    workdir=data_path,
)
mp.io.write_pdb(data_path / "stp1_react.pdb", stp1_react.to_frame())

In [ ]:
init1 = stp1_react.atoms[0]
init2 = stp1_react.atoms[16]
end1 = stp1_react.atoms[4]
end2 = stp1_react.atoms[12]
del1 = stp1_react.atoms[12]
del2 = stp1_react.atoms[12]

stp2_unreact = reacter.find_template(
    "stp2_unreact",
    stp1_react,
    end1, end2
)

stp2_react = reacter.react(
    "stp2_react",
    stp2_unreact,
    init1, init2,
    end1, end2,
    [del1, del2],
    workdir=data_path,
)
mp.io.write_pdb(data_path / "stp2_react.pdb", stp1_react.to_frame())